In [28]:
import re
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
data=pd.read_csv(r"C:\Users\hp\Downloads\0000000000002747_training_twitter_x_y_train.csv")
data_test=pd.read_csv(r"C:\Users\hp\Downloads\0000000000002747_test_twitter_x_test.csv")
df=data

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [29]:
df.shape

(10980, 12)

In [30]:
l=[]
for i in df['airline_sentiment']:
    if(i=='negative'):
        l.append(-1)
        
    elif(i=='neutral'):
        l.append(0)
        
    else:
        l.append(1)
        
y_train=np.array(l)        

In [31]:
df = data.append(data_test, ignore_index=True)

In [32]:
df     

,airline,airline_sentiment,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,Southwest,negative,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,567900433542488064,Washington D.C.,Atlantic Time (Canada)
1,Southwest,positive,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,569989168903819264,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,United,positive,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,568089179520954368,Illinois,Central Time (US & Canada)
3,Southwest,negative,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,568928195581513728,NaN,Atlantic Time (Canada)
4,United,negative,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,568594180014014464,NaN,Eastern Time (US & Canada)
5,United,negative,NaN,bobgiolito,NaN,0,@united Why did you load us in this flying sar...,NaN,2015-02-22 17:58:27 -0800,569677636613439488,"Los Angeles, CA",Pacific Time (US & Canada)
6,Delta,negative,NaN,aaronkinnari,NaN,0,@JetBlue that is a stock response. Delays not ...,NaN,2015-02-22 16:44:00 -0800,569658903044218880,Gotham,Quito
7,Delta,positive,NaN,TimothySays,NaN,0,@JetBlue That'd be nice! Hoping to rack up eno...,NaN,2015-02-19 14:48:53 -0800,568542766860541952,"Burlington, MA",Eastern Time (US & Canada)
8,United,negative,NaN,lindaSWC,NaN,1,@united frankly worse customer service ever. P...,NaN,2015-02-23 23:01:11 -0800,570116209263427584,NaN,NaN
9,Southwest,positive,NaN,amyums,NaN,0,@SouthwestAir yeah haha. Never been in one. It...,NaN,2015-02-20 12:29:46 -0800,568870144891600896,NaN,Central Time (US & Canada)


In [33]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt  

In [34]:
dic=[]
for i in df.text:    
    dic.append(remove_pattern(i,"@[\w]*"))
     
df['rem_@']=dic

In [35]:
#df['tidy_tweet'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")

In [36]:
df['rem_@'] = df['rem_@'].str.replace("[^a-zA-Z]", " ")
df['rem_@'][24]

' good fly    United  businessFirst'

In [37]:
df['rem_@'] = df['rem_@'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [38]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop=stopwords.words('english')
import string
punctuations=list(string.punctuation)
stop=stop+punctuations

In [39]:
tokenized_tweet = df['rem_@'].apply(lambda x: word_tokenize(x))
tokenized_tweet[0:2]

0    [scheduled, morning, days, after, fact, sure, ...
1    [seeing, your, workers, time, time, going, abo...
Name: rem_@, dtype: object

In [40]:

tokenized_tweet=tokenized_tweet.apply(lambda m:[w for w in m if not w.lower() in stop])
tokenized_tweet[0:2]

0    [scheduled, morning, days, fact, sure, evening...
1    [seeing, workers, time, time, going, beyond, l...
Name: rem_@, dtype: object

In [41]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

In [42]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [43]:
from nltk.corpus import wordnet

In [44]:
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [45]:
tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i,pos=get_simple_pos(pos_tag([i])[0][1])) for i in x])
tokenized_tweet[0:3]

0    [schedule, morning, day, fact, sure, even, fli...
1    [see, worker, time, time, go, beyond, love, fl...
2    [Flew, Miami, back, great, crew, service, leg,...
Name: rem_@, dtype: object

In [46]:
type(tokenized_tweet[0])

list

In [47]:
all_word=[]
for doc in tokenized_tweet:
    all_word+=doc    

In [48]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df['rem_@'] = tokenized_tweet

In [49]:
all_words = ' '.join([text for text in df['rem_@']])

In [50]:
freq=nltk.FreqDist(all_word)
common=freq.most_common(3000)
feat=[i[0] for i in common]

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=10000)
x_train_features = count_vectorizer.fit_transform(df['rem_@'][0:10980])
x_train_features=x_train_features.toarray()

In [74]:
x_test_features=count_vectorizer.transform(df['rem_@'][10980:])
x_test_features=x_test_features.toarray()

In [75]:
type(df['airline_sentiment'][0])
df['airline_sentiment'][0]

'negative'

In [76]:
from sklearn.naive_bayes import GaussianNB
clf=GaussianNB()
from sklearn.linear_model import LogisticRegression
cl=LogisticRegression()
cl.fit(x_train_features,y_train)
ytest1=cl.predict(x_test_features)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [77]:
clf.fit(x_train_features,y_train)
ytest2=clf.predict(x_test_features)
#cl better

In [78]:
data_test['text'][1]

'@AmericanAir after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs.'

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf_train = tfidf_vectorizer.fit_transform(df['rem_@'][0:10980])

In [80]:
tfidf_test=tfidf_vectorizer.transform(df['rem_@'][10981:])

In [81]:
cl.fit(tfidf_train,y_train)
ytest3=cl.predict(tfidf_test)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [82]:
tfidf_train=tfidf_train.toarray()
tfidf_test=tfidf_test.toarray()
clf.fit(tfidf_train,y_train)
ytest4=clf.predict(tfidf_test)

In [83]:
y_test1=[]
for i in ytest1:
    if(i==-1):
        y_test1.append('negative')
        
    elif(i==0):
        y_test1.append('neutral')
        
    else:
        y_test1.append('positive')
        

        
y_test2=[]
for i in ytest2:
    if(i==-1):
        y_test2.append('negative')
        
    elif(i==0):
        y_test2.append('neutral')
        
    else:
        y_test2.append('positive')        
        
        
y_test3=[]
for i in ytest3:
    if(i==-1):
        y_test3.append('negative')
        
    elif(i==0):
        y_test3.append('neutral')
        
    else:
        y_test3.append('positive')
        
        
y_test4=[]
for i in ytest4:
    if(i==-1):
        y_test4.append('negative')
        
    elif(i==0):
        y_test4.append('neutral')
        
    else:
        y_test4.append('positive')        

In [84]:
np.savetxt("tts1.csv",y_test1,delimiter=",",fmt='%s')
np.savetxt("tts2.csv",y_test2,delimiter=",",fmt='%s')
np.savetxt("tts3.csv",y_test3,delimiter=",",fmt='%s')
np.savetxt("tts4.csv",y_test4,delimiter=",",fmt='%s')